## **Movielens-1M using PIML Xgboost User-Item** 
Compare the prediction with RankFM. Use XGBoost classification, regression and ranker.

#### **Data Loading and Processing**

In [56]:
import pandas as pd 
import numpy as np

from sklearn.model_selection import train_test_split

In [57]:
rating = pd.read_csv("ml-1m/ratings.dat", names=["user_id", "item_id", "rating", "timestamp"], delimiter="::", engine="python")
rating.columns = ['user_id','item_id','rating','timestamp']
rating

,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [58]:
unique_users = rating.user_id.nunique()
unique_items = rating.item_id.nunique()

print(f"interactions shape {rating.shape}")
print(f"interaction unique users {unique_users}")
print(f"interaction unique items {unique_items}")


interactions shape (1000209, 4)
interaction unique users 6040
interaction unique items 3706


In [59]:
sparsity = 1 - (len(rating)/(unique_users * unique_items))
print(f"interaction matrix sparsity: {round(100 * sparsity, 1)}%")

interaction matrix sparsity: 95.5%


In [60]:
rating.drop(['timestamp'], axis=1, inplace=True)

# set all positive interactions to 1
df_classification = rating.copy()
df_classification['interaction'] = 1
df_classification.drop(['rating'], axis=1, inplace=True)

#### **Classification Approach**
in RankFM, all dataset is positive interaction. To make it as classification prediction, balanced synthesis negative interaction added

In [61]:
import random

all_users = df_classification.user_id.unique()
all_items = df_classification.item_id.unique()

negative_instances = []

for user in all_users:
    user_interacted_item = df_classification[df_classification.user_id == user]['item_id'].unique()
    non_interacted_items = set(all_items) - set(user_interacted_item)
    for item in non_interacted_items:
        negative_instances.append([user, item, 0])

num_negatives = len(rating[df_classification['interaction'] == 1])
sampled_negatives = random.sample(negative_instances, num_negatives)

df_negatives = pd.DataFrame(sampled_negatives, columns=['user_id', 'item_id', 'interaction'])
df_negatives

columns = ['user_id', 'item_id', 'interaction']
balanced_df = pd.concat([df_classification[columns], df_negatives[columns]]).reset_index(drop=True)

In [62]:
df = balanced_df

#### Data Preparation with PIML

In [63]:
from piml import Experiment
from piml.models import XGB2Regressor, XGB2Classifier
from sklearn.model_selection import train_test_split

user_item = df[['user_id', 'item_id']]

# Prepare the features and labels
X = df.drop(['interaction'], axis=1)
y = df['interaction']

# Perform train-test split
train_x, test_x, train_y, test_y, user_item_train, user_item_test = train_test_split(
    X, y, user_item, test_size=0.25, random_state=42
)

# Get the indices for train and test sets
train_indices = np.array(train_x.index)
test_indices = np.array(test_x.index)

exp = Experiment(highcode_only=True)
exp.data_loader(data=df, silent=True)
exp.data_summary(feature_exclude=[], silent=True)
exp.data_prepare(target='interaction', task_type='classification', train_idx=train_indices, test_idx=test_indices, silent=True)
# exp.feature_select(threshold=0.98, method="pfi", figsize=(6, 5))


In [64]:
print("train shape: {}".format(train_x.shape))
print("valid shape: {}".format(test_x.shape))

train_users = np.sort(user_item_train.user_id.unique())
valid_users = np.sort(user_item_test.user_id.unique())
cold_start_users = set(valid_users) - set(train_users)

train_items = np.sort(user_item_train.item_id.unique())
valid_items = np.sort(user_item_test.item_id.unique())
cold_start_items = set(valid_items) - set(train_items)

print("train users: {}".format(len(train_users)))
print("valid users: {}".format(len(valid_users)))
print("cold-start users: {}".format(cold_start_users))

print("train items: {}".format(len(train_items)))
print("valid items: {}".format(len(valid_items)))
print("cold-start items: {}".format(cold_start_items))

train shape: (1500313, 2)
valid shape: (500105, 2)
train users: 6040
valid users: 6040
cold-start users: set()
train items: 3706
valid items: 3706
cold-start items: set()


#### Model Training

In [65]:
exp.model_train(model=XGB2Classifier(), name="XGB2")
exp.model_diagnose(model="XGB2", show="accuracy_table")

,ACC,AUC,F1,LogLoss,Brier
,,,,,
Train,0.6100,0.6563,0.5987,0.6569,0.2325
Test,0.6097,0.6566,0.5994,0.6568,0.2324
Gap,-0.0003,0.0003,0.0007,-0.0001,-0.0001


In [66]:
# exp.model_interpret(model="XGB2", show="global_ei", figsize=(5, 4))


In [67]:
# exp.model_interpret(model="XGB2", show="global_fi", figsize=(5, 4))

In [68]:
# exp.model_interpret(model="XGB2", show="local_ei", sample_id=0, original_scale=True, figsize=(5, 4))

In [69]:
# exp.model_interpret(model="XGB2", show="local_fi", sample_id=0, original_scale=True, figsize=(5, 4))


#### Use RankFM Validation Data

In [70]:
np.random.seed(42)

interactions = rating.copy()
interactions['random'] = np.random.random(size=len(interactions))
test_pct = 0.25

train_mask = interactions['random'] <  (1 - test_pct)
valid_mask = interactions['random'] >= (1 - test_pct)

interactions_train = interactions[train_mask][['user_id', 'item_id']]
interactions_valid = interactions[valid_mask][['user_id', 'item_id']]

train_users = np.sort(interactions_train.user_id.unique())
valid_users = np.sort(interactions_valid.user_id.unique())
cold_start_users = set(valid_users) - set(train_users)

train_items = np.sort(interactions_train.item_id.unique())
valid_items = np.sort(interactions_valid.item_id.unique())
cold_start_items = set(valid_items) - set(train_items)

print("train shape: {}".format(interactions_train.shape))
print("valid shape: {}".format(interactions_valid.shape))

print("train users: {}".format(len(train_users)))
print("valid users: {}".format(len(valid_users)))
print("cold-start users: {}".format(cold_start_users))

print("train items: {}".format(len(train_items)))
print("valid items: {}".format(len(valid_items)))
print("cold-start items: {}".format(cold_start_items))


train shape: (750042, 2)
valid shape: (250167, 2)
train users: 6040
valid users: 6040
cold-start users: set()
train items: 3670
valid items: 3507
cold-start items: {3842, 2308, 2438, 3220, 3607, 2584, 1820, 2845, 2591, 545, 1316, 2214, 1832, 1579, 3376, 1714, 1843, 2226, 2742, 311, 826, 2235, 3517, 1470, 576, 2895, 601, 3291, 989, 1630, 2909, 868, 2277, 2039, 3065, 2556}


#### Predict on test data  
Make the top-10 K 

In [71]:
k = 10

def pivot_table_recommendation(recommendation):
    # Pivot the data to get a wide format DataFrame with one row per user and top 10 movie recommendations
    pv_rec = recommendation.pivot(index='user_id', columns='rank', values='item_id').reset_index()

    # Set user_id as the index
    pv_rec.set_index('user_id', inplace=True)

    # Remove the 'rank' column
    pv_rec.columns.name = None
    pv_rec.columns = [f'{int(rank)}' for rank in pv_rec.columns]
    pv_rec.index.name = None

    return pv_rec

#### Predict on Test Dataset

In [72]:
df_test = user_item_test
dpredict = df_test

xgb2 = exp.get_model("XGB2")

# Predict the interaction probabilities
df_test['predicted_interaction'] = xgb2.predict(dpredict)

# Rank the predictions
df_test['rank'] = df_test.groupby('user_id')['predicted_interaction'].rank(method='first', ascending=False)

# Filter to get top 10 predictions for each user
top_10_recommendations = df_test[df_test['rank'] <= 10]

# Pivot the data to get a wide format DataFrame with one row per user and top 10 movie recommendations
pivot_recommendations = pivot_table_recommendation(top_10_recommendations)
pivot_recommendations.head()

,1,2,3,4,5,6,7,8,9,10
1,2027,921,1246,1306,353,2692,1101,3268,1207,1
2,366,21,1253,356,1197,1351,1207,1245,2427,1198
3,1259,1394,2400,2921,1995,1251,1088,372,42,2507
4,1954,1349,1238,1187,1036,1214,1391,373,1386,3802
5,593,2427,1095,32,1218,1335,2428,1392,916,1089


**Notes:** 
- RankFM able to generate movie_id based on user_id input, while XGBoost is predict the interaction given user_id and movie_id

#### Predict on RankFM Validation dataset

In [73]:
df_test = interactions_valid.copy()

dpredict = df_test

xgb2 = exp.get_model("XGB2")

# Predict the interaction probabilities
df_test['predicted_interaction'] = xgb2.predict(dpredict)

# Rank the predictions
df_test['rank'] = df_test.groupby('user_id')['predicted_interaction'].rank(method='first', ascending=False)

# Filter to get top 10 predictions for each user
top_10_recommendations = df_test[df_test['rank'] <= 10]

# Pivot the data to get a wide format DataFrame with one row per user and top 10 movie recommendations
pivot_recommendations = pivot_table_recommendation(top_10_recommendations)
pivot_recommendations.head()

,1,2,3,4,5,6,7,8,9,10
1,2398.0,48.0,2692.0,1246.0,661.0,2804.0,938.0,588.0,1907.0,783.0
2,1357.0,2916.0,1213.0,1293.0,3257.0,2028.0,1084.0,1265.0,1193.0,590.0
3,1079.0,1580.0,1968.0,3421.0,1641.0,3534.0,3868.0,653.0,2167.0,3114.0
4,1196.0,1198.0,1387.0,480.0,3418.0,2366.0,3527.0,2947.0,NaN,NaN
5,1392.0,1191.0,2427.0,908.0,265.0,2029.0,1089.0,913.0,3266.0,2395.0


**Notes:** 
- The results is looks totally differences than RankFM and few is NaN

In [74]:
k = 10

test_users_items = df_test.groupby('user_id')['item_id'].apply(set).to_dict()
test_users = list(test_users_items.keys())
comm_user = pivot_recommendations.index.values

def hit_rate():
    hit_rate = np.mean([int(len(set(pivot_recommendations.loc[u]) & test_users_items[u]) > 0) for u in comm_user])
    return hit_rate

def reciprocal_rank():
    match_indexes = [np.where(pivot_recommendations.loc[u].isin(set(pivot_recommendations.loc[u]) & test_users_items[u]))[0] for u in comm_user]
    reciprocal_rank = np.mean([1 / (np.min(index) + 1) if len(index) > 0 else 0 for index in match_indexes])

    return reciprocal_rank

def dcg():
    match_indexes = [np.where(pivot_recommendations.loc[u].isin(set(pivot_recommendations.loc[u]) & test_users_items[u]))[0] for u in comm_user]
    discounted_cumulative_gain = np.mean([np.sum(1 / np.log2(index + 2)) if len(index) > 0 else 0 for index in match_indexes])
    
    return discounted_cumulative_gain

def precision():
    precision = np.mean([len(set(pivot_recommendations.loc[u]) & test_users_items[u]) / len(pivot_recommendations.loc[u]) for u in comm_user])

    return precision

def recall():
    recall = np.mean([len(set(pivot_recommendations.loc[u]) & test_users_items[u]) / len(test_users_items[u]) for u in comm_user])

    return recall

print("EVALUATION CLASSIFICATION TESTSET ONLY\n")

print("hit_rate: {:.3f}".format(hit_rate()))
print("reciprocal_rank: {:.3f}".format(reciprocal_rank()))
print("dcg: {:.3f}".format(dcg()))
print("precision: {:.3f}".format(precision()))
print("recall: {:.3f}".format(recall()))

EVALUATION CLASSIFICATION TESTSET ONLY

hit_rate: 1.000
reciprocal_rank: 1.000
dcg: 4.315
precision: 0.928
recall: 0.510


**Notes:** 

- XGB2 outperform because its leverage rich data from user and item features
- Next, we test on unseen data to check whether its over-fitting

#### Prediction on All Combination Data (excluded training data)

In [75]:
# Assuming all_users is a list of all user IDs and all_items is a list of all item IDs
# Create a DataFrame for all user-item pairs
all_user_item_pairs = pd.DataFrame([(user, item) for user in all_users for item in all_items], columns=['user_id', 'item_id'])

# Remove training data from all_user_item_pairs
all_user_item_pairs = all_user_item_pairs[~all_user_item_pairs.isin(user_item_train[['user_id', 'item_id']])].dropna()

In [76]:
# Add user and item features
all_pairs_completed = all_user_item_pairs

dpredict = all_pairs_completed

xgb2 = exp.get_model("XGB2")

# Predict the interaction probabilities
all_pairs_completed['predicted_interaction'] = xgb2.predict(dpredict)

In [77]:
# Rank the predictions
all_pairs_completed['rank'] = all_pairs_completed.groupby('user_id')['predicted_interaction'].rank(method='first', ascending=False)

# Filter to get top 10 predictions for each user
top_10_recommendations = all_pairs_completed[all_pairs_completed['rank'] <= 10]

# Pivot the data to get a wide format DataFrame with one row per user and top 10 movie recommendations
pivot_recommendations = pivot_table_recommendation(top_10_recommendations)
pivot_recommendations.head()

,1,2,3,4,5,6,7,8,9,10
1.0,1.0,2692.0,1207.0,1246.0,1357.0,1213.0,1253.0,902.0,368.0,1259.0
2.0,1193.0,914.0,1197.0,1287.0,594.0,919.0,595.0,2398.0,2918.0,1035.0
3.0,1193.0,914.0,1197.0,1287.0,594.0,919.0,595.0,2398.0,2918.0,1035.0
4.0,1193.0,914.0,1197.0,1287.0,594.0,919.0,595.0,2398.0,2918.0,1035.0
5.0,1193.0,914.0,1197.0,1287.0,594.0,919.0,595.0,2398.0,2918.0,1035.0


#### Evaluation Unseen Data Prediction Results

In [78]:
test_users_items = df_test.groupby('user_id')['item_id'].apply(set).to_dict()
test_users = list(test_users_items.keys())
comm_user = pivot_recommendations.index.values

print("EVALUATION CLASSIFICATION UNSEEN DATA EXCLUDED TRAINING\n")

print("hit_rate: {:.3f}".format(hit_rate()))
print("reciprocal_rank: {:.3f}".format(reciprocal_rank()))
print("dcg: {:.3f}".format(dcg()))
print("precision: {:.3f}".format(precision()))
print("recall: {:.3f}".format(recall()))

EVALUATION CLASSIFICATION UNSEEN DATA EXCLUDED TRAINING

hit_rate: 0.367
reciprocal_rank: 0.139
dcg: 0.249
precision: 0.053
recall: 0.017


**Notes:** 

- Its significant lower when predicting over unseen data and back testing into test data

#### **Regression Approach**

In [79]:
df = rating

user_item = df[['user_id', 'item_id']]

# Prepare the features and labels
X = df.drop(['rating'], axis=1)
y = df['rating']

# Perform train-test split
train_x, test_x, train_y, test_y, user_item_train, user_item_test = train_test_split(
    X, y, user_item, test_size=0.25, random_state=42
)

# Get the indices for train and test sets
train_indices = np.array(train_x.index)
test_indices = np.array(test_x.index)

exp = Experiment(highcode_only=True)
exp.data_loader(data=df, silent=True)
exp.data_summary(feature_exclude=['rating'], silent=True)
exp.data_prepare(target='rating', task_type='regression', train_idx=train_indices, test_idx=test_indices, silent=True)
# exp.feature_select(threshold=0.98, method="pfi", figsize=(6, 5))

In [80]:
exp.model_train(model=XGB2Regressor(), name="XGB2")
exp.model_diagnose(model="XGB2", show="accuracy_table")

,MSE,MAE,R2
,,,
Train,0.0723,0.2212,0.0717
Test,0.0728,0.2220,0.0697
Gap,0.0005,0.0008,-0.0020


In [81]:
df_test = user_item_test

dpredict = df_test

xgb2 = exp.get_model("XGB2")

# Predict the interaction probabilities
df_test['predicted_rating'] = xgb2.predict(dpredict)

# Rank the predictions
df_test['rank'] = df_test.groupby('user_id')['predicted_rating'].rank(method='first', ascending=False)

# Filter to get top 10 predictions for each user
top_10_recommendations = df_test[df_test['rank'] <= 10]

# Pivot the data to get a wide format DataFrame with one row per user and top 10 movie recommendations
pivot_recommendations = pivot_table_recommendation(top_10_recommendations)
pivot_recommendations.head()

,1,2,3,4,5,6,7,8,9,10
1,1193.0,1246.0,595.0,1035.0,608.0,588.0,3105.0,3186.0,2321.0,2340.0
2,1253.0,1196.0,265.0,1084.0,1090.0,110.0,1945.0,1957.0,1953.0,3678.0
3,1196.0,1259.0,1270.0,1291.0,1049.0,3671.0,3168.0,2081.0,1394.0,653.0
4,1210.0,1201.0,260.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,913.0,919.0,1279.0,1192.0,1175.0,1191.0,1171.0,321.0,593.0,1089.0


#### Test with RankFM Validation Dataset

In [82]:
df_test = interactions_valid.copy()
dpredict = df_test

xgb2 = exp.get_model("XGB2")

# Predict the interaction probabilities
df_test['predicted_rating'] = xgb2.predict(dpredict)

# Rank the predictions
df_test['rank'] = df_test.groupby('user_id')['predicted_rating'].rank(method='first', ascending=False)

# Filter to get top 10 predictions for each user
top_10_recommendations = df_test[df_test['rank'] <= 10]

# Pivot the data to get a wide format DataFrame with one row per user and top 10 movie recommendations
pivot_recommendations = pivot_table_recommendation(top_10_recommendations)
pivot_recommendations.head()

,1,2,3,4,5,6,7,8,9,10
1,1246.0,938.0,48.0,608.0,588.0,3114.0,2804.0,1907.0,2692.0,661.0
2,1213.0,1193.0,1196.0,1245.0,920.0,1265.0,1293.0,590.0,1084.0,1953.0
3,1079.0,2871.0,3671.0,3114.0,3534.0,1641.0,3421.0,3552.0,2167.0,653.0
4,1196.0,1198.0,2947.0,3527.0,2366.0,3418.0,480.0,1387.0,NaN,NaN
5,908.0,913.0,265.0,1175.0,1191.0,1171.0,860.0,321.0,1089.0,2395.0


In [84]:
test_users_items = df_test.groupby('user_id')['item_id'].apply(set).to_dict()
test_users = list(test_users_items.keys())
comm_user = pivot_recommendations.index.values

print("EVALUATION REGRESSION TEST SET ONLY \n")

print("hit_rate: {:.3f}".format(hit_rate()))
print("reciprocal_rank: {:.3f}".format(reciprocal_rank()))
print("dcg: {:.3f}".format(dcg()))
print("precision: {:.3f}".format(precision()))
print("recall: {:.3f}".format(recall()))

EVALUATION REGRESSION TEST SET ONLY 

hit_rate: 1.000
reciprocal_rank: 1.000
dcg: 4.315
precision: 0.928
recall: 0.510


In [85]:
# Assuming all_users is a list of all user IDs and all_items is a list of all item IDs
# Create a DataFrame for all user-item pairs
all_user_item_pairs = pd.DataFrame([(user, item) for user in rating.user_id.unique() for item in rating.item_id.unique()], columns=['user_id', 'item_id'])

# Remove training data from all_user_item_pairs
all_user_item_pairs = all_user_item_pairs[~all_user_item_pairs.isin(user_item_train[['user_id', 'item_id']])].dropna()

all_pairs_completed = all_user_item_pairs.copy()

In [86]:
xgb2 = exp.get_model("XGB2")

# Predict the interaction probabilities
all_user_item_pairs['predicted_rating'] = xgb2.predict(all_pairs_completed)

In [87]:
# Rank the predictions
all_user_item_pairs['rank'] = all_user_item_pairs.groupby('user_id')['predicted_rating'].rank(method='first', ascending=False)

# Filter to get top 10 predictions for each user
top_10_recommendations = all_user_item_pairs[all_user_item_pairs['rank'] <= 10]

# Pivot the data to get a wide format DataFrame with one row per user and top 10 movie recommendations
pivot_recommendations = pivot_table_recommendation(top_10_recommendations)
pivot_recommendations.head()

,1,2,3,4,5,6,7,8,9,10
1.0,1193.0,1246.0,1253.0,1225.0,1196.0,1198.0,1244.0,1245.0,1247.0,1214.0
2.0,1193.0,1197.0,1207.0,1246.0,1210.0,1213.0,1217.0,1253.0,1225.0,1196.0
3.0,1193.0,1197.0,1207.0,1246.0,1210.0,1213.0,1217.0,1253.0,1225.0,1196.0
4.0,1193.0,1197.0,1207.0,1246.0,1210.0,1213.0,1217.0,1253.0,1225.0,1196.0
5.0,1193.0,1197.0,1207.0,1246.0,1210.0,1213.0,1217.0,1253.0,1225.0,1196.0


In [88]:
test_users_items = df_test.groupby('user_id')['item_id'].apply(set).to_dict()
test_users = list(test_users_items.keys())
comm_user = pivot_recommendations.index.values

print("EVALUATION REGRESSION UNSEEN DATA EXCLUDED TRAINING \n")

print("hit_rate: {:.3f}".format(hit_rate()))
print("reciprocal_rank: {:.3f}".format(reciprocal_rank()))
print("dcg: {:.3f}".format(dcg()))
print("precision: {:.3f}".format(precision()))
print("recall: {:.3f}".format(recall()))

EVALUATION REGRESSION UNSEEN DATA EXCLUDED TRAINING 

hit_rate: 0.432
reciprocal_rank: 0.163
dcg: 0.301
precision: 0.065
recall: 0.023


#### **XGBRanker Approach**
We are using the classification data that have positive and negative balance

In [89]:
df = balanced_df.copy()

In [90]:
from sklearn.model_selection import GroupShuffleSplit

gss = GroupShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
train_idx, test_indices = next(gss.split(df, groups=df.user_id))

train_data = df.iloc[train_idx]
test_data = df.iloc[test_indices]

X_train = train_data.drop(columns=['interaction'])
y_train = train_data['interaction']
X_test = test_data.drop(columns=['interaction'])
y_test = test_data['interaction']


In [91]:
train_groups = X_train.groupby('user_id').size().to_numpy()
test_groups = X_test.groupby('user_id').size().to_numpy()

In [92]:
import xgboost as xgb 

dtrain = xgb.DMatrix(X_train, label=y_train)
dtrain.set_group(train_groups)

dtest = xgb.DMatrix(X_test, label=y_test)
dtest.set_group(test_groups)

params = {
    'objective': 'rank:pairwise',
    'eval_metric': 'ndcg',
    'learning_rate': 0.1,
    'max_depth': 6,
    'verbose': 0 
}

bst = xgb.train(params, dtrain, num_boost_round=100, evals=[(dtest, 'test')], early_stopping_rounds=10)

[0]	test-ndcg:1.00000
[1]	test-ndcg:1.00000
[2]	test-ndcg:1.00000
[3]	test-ndcg:1.00000
[4]	test-ndcg:1.00000
[5]	test-ndcg:1.00000
[6]	test-ndcg:1.00000
[7]	test-ndcg:1.00000
[8]	test-ndcg:1.00000
[9]	test-ndcg:1.00000


In [93]:
test_data['predicted_score'] = bst.predict(dtest)
test_data['rank'] = test_data.groupby('user_id')['predicted_score'].rank(method='first', ascending=False)

top_10_recommendations = test_data[test_data['rank'] <= 10]

# Pivot the data to get a wide format DataFrame with one row per user and top 10 movie recommendations
pivot_recommendations = pivot_table_recommendation(top_10_recommendations)
pivot_recommendations.head()

,1,2,3,4,5,6,7,8,9,10
9,2268,1466,1393,861,1682,3717,508,3793,720,367
13,2987,648,2628,2054,1259,589,1690,2,153,1331
15,3421,648,3354,2485,141,2126,2058,3798,2997,653
16,2987,2555,2629,1682,2485,2701,2568,3004,1269,2713
18,2987,2989,2622,648,2628,1682,1683,1688,2123,2052


In [94]:
test_users_items = df_test.groupby('user_id')['item_id'].apply(set).to_dict()
test_users = list(test_users_items.keys())
comm_user = pivot_recommendations.index.values

print("EVALUATION XGB RANKER ON TESTSET\n")

print("hit_rate: {:.3f}".format(hit_rate()))
print("reciprocal_rank: {:.3f}".format(reciprocal_rank()))
print("dcg: {:.3f}".format(dcg()))
print("precision: {:.3f}".format(precision()))
print("recall: {:.3f}".format(recall()))

EVALUATION XGB RANKER ON TESTSET

hit_rate: 0.942
reciprocal_rank: 0.465
dcg: 1.138
precision: 0.248
recall: 0.157


#### **Result Comparison**

| Metrics | Classification (all-unseen) | Regressor (all-unseen) | Rank Based | RankFM |
| --- | --- | --- | --- | --- | 
| hit_rate |  0.367 | 0.432 | 0.942 | 0.788 |
| reciprocal_rank | 0.139 | 0.163 | 0.465 | 0.334 |
| dcg | 0.249 |  0.301 | 1.138 | 0.718 |
| precision | 0.053 | 0.065 | 0.248 | 0.156 |
| recall | 0.017 | 0.023 | 0.157 | 0.072 |